## Načítanie knižníc a trénovacieho datasetu

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import scipy.stats as ss
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import seaborn as sns
import math

In [2]:
df = pd.read_parquet("ml_dataset_train.parquet")

In [3]:
df.head()

,account_id,n_ad_reward_claims,n_ad_reward_fails,n_ads_watched,n_battlepass_lvls_finished,masked_feature_0,masked_feature_1,masked_feature_2,masked_feature_3,masked_feature_4,...,battlepass_22.0,battlepass_23.0,battlepass_8008.0,first_login_day_time,first_login_country_is_mfreq,time_to_first_purchase,time_to_last_purchase,time_between_last_purchase_last_login,sum_payments_package_key_ltv,n_payments_package_key_ltv
0,100000042,13,<NA>,26,8,175,354.0,354,281.0,141,...,False,False,False,eve,1,NaN,NaN,NaN,NaN,<NA>
1,100000053,<NA>,<NA>,7,1,25,17.0,17,8.0,8,...,False,False,False,morning,1,NaN,NaN,NaN,NaN,<NA>
2,100000082,<NA>,<NA>,<NA>,<NA>,1,NaN,<NA>,NaN,<NA>,...,False,False,False,eve,1,NaN,NaN,NaN,NaN,<NA>
3,100000112,9,<NA>,36,5,72,197.0,197,194.0,83,...,False,False,False,eve,1,NaN,NaN,NaN,NaN,<NA>
4,100000122,<NA>,<NA>,1,3,67,108.0,108,96.0,38,...,False,False,False,eve,1,NaN,NaN,NaN,NaN,<NA>


In [4]:
df.loc[:,["account_id","n_ads_watched","first_login_day_time","sum_payments_package_key","sum_payments_package_key_ltv"]].sort_values(by="sum_payments_package_key_ltv",ascending=False).head()

,account_id,n_ads_watched,first_login_day_time,sum_payments_package_key,sum_payments_package_key_ltv
2494459,169616730,3,early_morning,2854.17,22535.53
2508909,170207391,7,night,968.61,14256.66
2368471,165357101,1,noon,1446.35,11661.82
921994,127826834,<NA>,early_morning,984.62,7641.16
2145973,150404211,105,early_morning,63.89,6348.67


## Úprava datasetu a vyhodenie skrytých údajov(jednalo sa o citlivé údaje klientov)

In [5]:
empty_array = []
for i in range(0,42):
    empty_array.append("masked_feature_" +str(i))
print(empty_array)

['masked_feature_0', 'masked_feature_1', 'masked_feature_2', 'masked_feature_3', 'masked_feature_4', 'masked_feature_5', 'masked_feature_6', 'masked_feature_7', 'masked_feature_8', 'masked_feature_9', 'masked_feature_10', 'masked_feature_11', 'masked_feature_12', 'masked_feature_13', 'masked_feature_14', 'masked_feature_15', 'masked_feature_16', 'masked_feature_17', 'masked_feature_18', 'masked_feature_19', 'masked_feature_20', 'masked_feature_21', 'masked_feature_22', 'masked_feature_23', 'masked_feature_24', 'masked_feature_25', 'masked_feature_26', 'masked_feature_27', 'masked_feature_28', 'masked_feature_29', 'masked_feature_30', 'masked_feature_31', 'masked_feature_32', 'masked_feature_33', 'masked_feature_34', 'masked_feature_35', 'masked_feature_36', 'masked_feature_37', 'masked_feature_38', 'masked_feature_39', 'masked_feature_40', 'masked_feature_41']


In [6]:
df.drop(['masked_feature_0', 'masked_feature_1', 'masked_feature_2', 'masked_feature_3', 'masked_feature_4', 'masked_feature_5', 'masked_feature_6', 'masked_feature_7', 'masked_feature_8', 'masked_feature_9','masked_feature_13','masked_feature_14','masked_feature_15', 'masked_feature_16', 'masked_feature_17', 'masked_feature_18', 'masked_feature_20', 'masked_feature_21', 'masked_feature_22', 'masked_feature_23', 'masked_feature_24', 'masked_feature_25', 'masked_feature_26', 'masked_feature_27', 'masked_feature_28', 'masked_feature_29', 'masked_feature_30', 'masked_feature_31', 'masked_feature_32', 'masked_feature_33', 'masked_feature_34', 'masked_feature_35', 'masked_feature_36', 'masked_feature_37', 'masked_feature_38', 'masked_feature_39', 'masked_feature_40', 'masked_feature_41'],axis=1,inplace=True)

## Rozdelenie a úprava datasetu pre ďalší postup a popisná štatistika jednotlivých častí pre lepší prehľad

In [7]:
# rozdelenie datasetu na buyerov a non_buyerov
buyers = df.loc[(df["sum_payments_package_key_ltv"]>0)]
non_buyers = df.loc[(df["n_payments_package_key_ltv"].isna())]
buyers_count = len(buyers.index)
non_buyers_count = len(non_buyers.index)
buyers_non_buyers_ratio = round((buyers_count/non_buyers_count)*100,2)
print(f"Percento buyerov v datasete {buyers_non_buyers_ratio}")

Percento buyerov v datasete 2.55


In [8]:
#oddelenie numerických premenných pre buyerov od tých ostatných nakoľko v regresných modeloch budeme pracovať len s numerickými premennými
numeric_columns_buyers = buyers.dtypes[buyers.dtypes!=np.object_].index.tolist()
numeric_variables_buyers = buyers.loc[:,numeric_columns_buyers]
# nahradenie nan hodnôt nulov a zbavenie sa boolean hodnot+následne resetnutie indexu aby sme boli schopní pristupovať k dátam 
# cez index
numeric_variables_buyers.fillna(0,inplace = True)
numeric_variables_buyers.drop(["battlepass_0.0","battlepass_22.0","battlepass_23.0","battlepass_8008.0"],inplace=True,axis=1)
numeric_variables_buyers.reset_index(inplace=True,drop=True)

In [9]:
#popisná štatistika čiselných premenných pre buyerov
#numeric_variables_buyers.describe()

In [10]:
#Vyfiltrovanie textových premenných pre buyerov
string_columns_buyers = buyers.dtypes[buyers.dtypes==np.object_].index.tolist()
string_variables_buyers= buyers.loc[:,string_columns_buyers]

In [11]:
#popisná štatistika textových premenných pre buyerov
#string_variables_buyers.describe() 

In [12]:
#vyfiltrovanie číselných premenných pre non-buyerov
numeric_columns_non_buyers = non_buyers.dtypes[non_buyers.dtypes!=np.object_].index.tolist()
numeric_variables_non_buyers = non_buyers.loc[:,numeric_columns_non_buyers]

In [13]:
#popisná štatistika číselných premenných pre non-buyerov
#numeric_variables_non_buyers.describe()

In [14]:
# Vyfiltrovanie textových premenných pre non-buyerov
string_columns_non_buyers = non_buyers.dtypes[non_buyers.dtypes==np.object_].index.tolist()
string_variables_non_buyers = non_buyers.loc[:,string_columns_non_buyers]

In [15]:
# Popisná štatistika textových premenných pre non-buyerov
#string_variables_non_buyers.describe()

## Načítanie a úprava testovacieho datasetu

In [16]:
df_test = pd.read_parquet("ml_dataset_test.parquet")
df_test.drop(['masked_feature_0', 'masked_feature_1', 'masked_feature_2', 'masked_feature_3', 'masked_feature_4', 'masked_feature_5', 'masked_feature_6', 'masked_feature_7', 'masked_feature_8', 'masked_feature_9','masked_feature_13','masked_feature_14','masked_feature_15', 'masked_feature_16', 'masked_feature_17', 'masked_feature_18', 'masked_feature_20', 'masked_feature_21', 'masked_feature_22', 'masked_feature_23', 'masked_feature_24', 'masked_feature_25', 'masked_feature_26', 'masked_feature_27', 'masked_feature_28', 'masked_feature_29', 'masked_feature_30', 'masked_feature_31', 'masked_feature_32', 'masked_feature_33', 'masked_feature_34', 'masked_feature_35', 'masked_feature_36', 'masked_feature_37', 'masked_feature_38', 'masked_feature_39', 'masked_feature_40', 'masked_feature_41'],axis=1,inplace=True)
#df_test.head()

In [17]:
buyers_test = df_test.loc[(df_test["sum_payments_package_key_ltv"].notna())]
non_buyers_test = df_test.loc[(df_test["n_payments_package_key_ltv"].isna())]
buyers_count_test = len(buyers_test.index)
non_buyers_count_test = len(non_buyers_test.index)
buyers_non_buyers_ratio_test = round((buyers_count_test/non_buyers_count_test)*100,2)
buyers_non_buyers_ratio_test

2.55

In [18]:
#vyfiltrovanie numerickych premennych
buyers_test_numeric_columns = buyers_test.dtypes[buyers_test.dtypes!=np.object_].index.tolist()
buyers_test_numeric_variables = buyers_test.loc[:,buyers_test_numeric_columns]
# odstraneie boolean premenných pre zjednodušenie procesu a nahradenie nan honotami nulov + resetnutie indexu
buyers_test_numeric_variables.drop(["battlepass_0.0","battlepass_22.0","battlepass_23.0","battlepass_8008.0"],axis=1,inplace = True)
buyers_test_numeric_variables.fillna(0,inplace = True)
buyers_test_numeric_variables.reset_index(inplace=True,drop=True)
#buyers_test_numeric_variables.head() 

In [19]:
buyers_test_numeric_variables.describe()

,n_ad_reward_claims,n_ad_reward_fails,n_ads_watched,n_battlepass_lvls_finished,n_instant_awards_claims,n_extra_challenge_buys,connected_fb,connect_fb_attempt,n_friend_installs,n_accept_all_friends,...,screen_height,nunique_countries,nunique_network_types,nunique_iaps_bought,first_login_country_is_mfreq,time_to_first_purchase,time_to_last_purchase,time_between_last_purchase_last_login,sum_payments_package_key_ltv,n_payments_package_key_ltv
count,12347.000000,12347.000000,12347.000000,12347.000000,12347.000000,12347.000000,12347.000000,12347.000000,12347.000000,12347.000000,...,12347.000000,12347.000000,12347.000000,12347.000000,12347.000000,12347.000000,12347.000000,12347.000000,12347.000000,12347.000000
mean,5.166599,0.522637,35.175022,5.018142,0.113550,0.058395,0.199158,0.276180,0.005183,0.147323,...,917.080991,1.018628,1.778894,2.130315,0.995869,47926.102443,79903.322838,128830.800572,39.847188,4.815097
std,6.161249,1.743712,47.612478,2.982235,0.504861,0.335889,0.473780,0.674465,0.080330,0.692407,...,1063.721826,0.148897,0.606032,2.439206,0.064139,62810.690690,79620.244017,83704.420949,198.594481,10.309769
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.990000,1.000000
25%,0.000000,0.000000,4.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,1.000000,1.000000,1.000000,1.000000,3759.505000,7336.701500,54869.320500,1.995000,1.000000
50%,4.000000,0.000000,17.000000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,1.000000,2.000000,1.000000,1.000000,14856.267000,52145.355000,134973.114000,4.990000,2.000000
75%,9.000000,0.000000,47.000000,7.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2009.000000,1.000000,2.000000,2.000000,1.000000,75674.874500,140535.069500,204529.020000,22.970000,4.000000
max,44.000000,39.000000,635.000000,50.000000,10.000000,8.000000,11.000000,13.000000,3.000000,16.000000,...,3200.000000,4.000000,5.000000,47.000000,1.000000,259119.486000,259197.446000,258530.907000,11701.320000,368.000000


## Lineárna regresia za pomoci scikit learn

In [20]:
X_train = numeric_variables_buyers.iloc[:,:46]
Y_train = numeric_variables_buyers.iloc[:,46].to_frame()

In [21]:
X_test = buyers_test_numeric_variables.iloc[:,:46]
Y_test = buyers_test_numeric_variables.iloc[:,46].to_frame()

In [22]:
regr = linear_model.LinearRegression(fit_intercept=True)
regr.fit(X_train,Y_train)
y_pred = regr.predict(X_test)

In [23]:
#score parameters
print(f"R^2 skóre pre tréningový dataset: {regr.score(X_train,Y_train)}")
print(f"R^2 skóre pre testovací dataet: {regr.score(X_test,Y_test)}")
print(f"MAE: {mean_absolute_error(Y_test,y_pred)}")
print(f"MSE: {mean_squared_error(Y_test,y_pred)}")

R^2 skóre pre tréningový dataset: 0.42910973974527045
R^2 skóre pre testovací dataet: 0.32150175105689804
MAE: 37.425503741684906
MSE: 26757.646091600756


In [24]:
#list1 = regr.coef_
#list1 = list1.reshape()
#list1 = np.reshape(list1,newshape=(46,1))
#list1 = list1.tolist()
#list2 = X_train.columns.tolist()
#list3 = [item for sublist in zip(list2, list1) for item in sublist]

#print(list3)
               

In [25]:
#print("Coefficient of determination: %.2f" % r2_score(Y_test, y_pred))

## Lineárna regresia za pomoci statsmodels

In [26]:
#dep_var = Y_train
#exp_var = X_train
#exp_var = sm.add_constant(exp_var) bola aj tak nevyznamna a zvysilo to kvalitu modelu plus umoznilo robit predict cez tuto kniznicu

In [27]:
#exp_var = sm.add_constant(exp_var)
#ols = sm.OLS(dep_var.astype(float),exp_var.astype(float))
#ols_fit = ols.fit()
#ols_fit.summary()

In [28]:
adjusted_X_train = X_train.loc[:,["n_battlepass_lvls_finished","n_extra_challenge_buys","connect_fb_attempt","n_accept_friend","n_clicked_add_friends","n_levels_completed","n_missions_completed","n_package_info_offers_viewed","sum_payments_package_key","n_package_tips_offers_viewed","n_sessions_ended","total_session_duration","n_sessions_started","min_session_start_player_level","n_time_skip_buys","n_viplevels_completed","gdp_per_capita","ram_max","avg_country_spend_30d","avg_country_spend_30d_first3d","nunique_iaps_bought","first_login_country_is_mfreq","time_to_first_purchase","time_between_last_purchase_last_login"]]
adjusted_X_test = X_test.loc[:,["n_battlepass_lvls_finished","n_extra_challenge_buys","connect_fb_attempt","n_accept_friend","n_clicked_add_friends","n_levels_completed","n_missions_completed","n_package_info_offers_viewed","sum_payments_package_key","n_package_tips_offers_viewed","n_sessions_ended","total_session_duration","n_sessions_started","min_session_start_player_level","n_time_skip_buys","n_viplevels_completed","gdp_per_capita","ram_max","avg_country_spend_30d","avg_country_spend_30d_first3d","nunique_iaps_bought","first_login_country_is_mfreq","time_to_first_purchase","time_between_last_purchase_last_login"]]

In [29]:
adj_dep_var = Y_train
adj_exp_var = adjusted_X_train

In [30]:
#adj_ols = sm.OLS(adj_dep_var.astype(float),adj_exp_var.astype(float))
#adj_ols_fit = adj_ols.fit()
#adj_ols_fit.summary()

## Random forrest regressor 

In [31]:
%%time

from sklearn.ensemble import RandomForestRegressor


rf = RandomForestRegressor(n_jobs=-1,n_estimators = 300,max_depth = 50,min_samples_split =5,min_samples_leaf= 2)
rf.fit(X_train, np.ravel(Y_train))
y_pred = rf.predict(X_test)

Wall time: 2min 30s


In [32]:
print(f"R^2 skóre pre trénovací dataset: {rf.score(X_train,Y_train)}")
print(f"R^2 skóre pre testovací dataset: {rf.score(X_test,Y_test)}")
print(f"MAE: {mean_absolute_error(Y_test,y_pred)}")
print(f"MSE: {mean_squared_error(Y_test,y_pred)}")
#mean_absolute_error(Y_test,y_pred_forrest)
#mean_squared_error(adj_dep_var,y_pred_forrest)

R^2 skóre pre trénovací dataset: 0.8113918074138069
R^2 skóre pre testovací dataset: 0.28907655083913764
MAE: 34.40956345118715
MSE: 28036.384884556566


## Logaritmická regresia

In [33]:
numeric_variables_buyers_log = []
for i in range(0,len(numeric_variables_buyers)):
    numeric_variables_buyers_log.append(math.log(numeric_variables_buyers.iloc[i,46]))

In [34]:
#fig, axes = plt.subplots(1, figsize=(15, 5), sharey=False,sharex=False)
#fig.suptitle('Distribúcia peňažných prostriedkov a frekvencie nákupov jednotlivých zákazníkov s využitím logaritmickej mierky')
#sns.set_theme(style="whitegrid",palette="colorblind")
#sns.histplot(x=numeric_variables_buyers_log,bins=10) # all buyers

In [35]:
numeric_variables_buyers_log_test = []
for i in range(0,len(buyers_test_numeric_variables)):
    numeric_variables_buyers_log_test.append(math.log(buyers_test_numeric_variables.iloc[i,46]))

In [36]:
log_y_train = {"sum_payments_package_key_ltv": numeric_variables_buyers_log}
log_y_train = pd.DataFrame(log_y_train)
#log_y_train

In [37]:
log_y_test = {"sum_payments_package_key_ltv": numeric_variables_buyers_log_test}
log_y_test = pd.DataFrame(log_y_test)
#log_y_test

In [38]:
log_regr = linear_model.LinearRegression()
log_regr.fit(X_train,log_y_train)
log_y_pred = log_regr.predict(X_test)

In [39]:
print(f"R^2 skóre pre trénovací dataset: {log_regr.score(X_train,log_y_train)}")
print(f"R^2 skóre pre testovací dataset: {log_regr.score(X_test,log_y_test)}")
print(f"MAE: {mean_absolute_error(log_y_test,log_y_pred)}")
print(f"MSE: {mean_squared_error(log_y_test,log_y_pred)}")

R^2 skóre pre trénovací dataset: 0.48585029363847165
R^2 skóre pre testovací dataset: 0.48660371078305575
MAE: 0.9118239533861466
MSE: 1.3644814340397158


In [40]:
dep_var = log_y_train
exp_var = X_train
exp_var = sm.add_constant(exp_var) 

In [41]:
ols = sm.OLS(dep_var.astype(float),exp_var.astype(float))
ols_fit = ols.fit()
ols_fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                 
========================================================================================
Dep. Variable:     sum_payments_package_key_ltv   R-squared:                       0.486
Model:                                      OLS   Adj. R-squared:                  0.486
Method:                           Least Squares   F-statistic:                     1466.
Date:                          Sun, 08 May 2022   Prob (F-statistic):               0.00
Time:                                  19:41:45   Log-Likelihood:            -1.1022e+05
No. Observations:                         69858   AIC:                         2.205e+05
Df Residuals:                             69812   BIC:                         2.210e+05
Df Model:                                    45                                         
Covariance Type:                      nonrobust                                         
=========================================================================================================
                                            coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------
const                                    -0.0996      0.097     -1.029      0.304      -0.290       0.090
n_ad_reward_claims                        0.0029      0.001      2.294      0.022       0.000       0.005
n_ad_reward_fails                        -0.0040      0.003     -1.486      0.137      -0.009       0.001
n_ads_watched                            -0.0012      0.000     -9.815      0.000      -0.001      -0.001
n_battlepass_lvls_finished                0.0476      0.004     13.073      0.000       0.040       0.055
n_instant_awards_claims                  -0.0100      0.011     -0.947      0.344      -0.031       0.011
n_extra_challenge_buys                   -0.0805      0.013     -6.006      0.000      -0.107      -0.054
connected_fb                             -0.0206      0.016     -1.250      0.211      -0.053       0.012
connect_fb_attempt                        0.0239      0.012      1.998      0.046       0.000       0.047
n_friend_installs                        -0.0539      0.053     -1.022      0.307      -0.157       0.049
n_accept_all_friends                     -0.0169      0.007     -2.395      0.017      -0.031      -0.003
n_accept_friend                          -0.0552      0.010     -5.696      0.000      -0.074      -0.036
n_clicked_add_friends                    -0.0138      0.004     -3.535      0.000      -0.021      -0.006
n_friend_searches                         0.0196      0.018      1.071      0.284      -0.016       0.055
n_levels_completed                       -0.0002      0.002     -0.116      0.908      -0.004       0.003
n_calendar_login_days                     0.0006      0.001      1.033      0.302      -0.001       0.002
n_missions_completed                      0.0003   8.28e-05      3.425      0.001       0.000       0.000
n_package_info_offers_viewed             -0.0001   9.78e-06    -11.395      0.000      -0.000   -9.22e-05
n_payments_package_key                   -0.0195      0.006     -3.419      0.001      -0.031      -0.008
sum_payments_package_key                  0.0040      0.000     24.182      0.000       0.004       0.004
n_package_tips_offers_viewed              0.0011      0.000      3.904      0.000       0.001       0.002
rated_game                               -0.0076      0.013     -0.584      0.559      -0.033       0.018
n_remove_ad_clicks                        0.0144      0.007      2.160      0.031       0.001       0.028
n_sessions_ended                         -0.0060      0.001     -4.268      0.000      -0.009      -0.003
total_session_duration                 5.519e-09   3.55e-10     15.532      0.000    4.82e-09    6.22e-09
max_s

In [44]:
#OLS regresia po vyhodení nevýznamných premenných vrátane koeficientu
dep_var = log_y_train
exp_var = X_train.drop(["n_ad_reward_fails","n_instant_awards_claims","connected_fb","n_friend_searches","n_levels_completed","n_calendar_login_days","rated_game","max_session_end_player_level","screen_density","avg_country_spend_30d","n_friend_installs"],axis=1)
exp_var = sm.add_constant(exp_var) #bola aj tak nevyznamna a zvysilo to kvalitu modelu plus umoznilo robit predict cez tuto kniznicu

In [45]:
ols = sm.OLS(dep_var.astype(float),exp_var.astype(float))
ols_fit = ols.fit()
ols_fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                 
========================================================================================
Dep. Variable:     sum_payments_package_key_ltv   R-squared:                       0.486
Model:                                      OLS   Adj. R-squared:                  0.486
Method:                           Least Squares   F-statistic:                     1940.
Date:                          Sun, 08 May 2022   Prob (F-statistic):               0.00
Time:                                  19:42:53   Log-Likelihood:            -1.1023e+05
No. Observations:                         69858   AIC:                         2.205e+05
Df Residuals:                             69823   BIC:                         2.208e+05
Df Model:                                    34                                         
Covariance Type:                      nonrobust                                         
=========================================================================================================
                                            coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------
const                                    -0.0834      0.096     -0.865      0.387      -0.272       0.106
n_ad_reward_claims                        0.0032      0.001      2.561      0.010       0.001       0.006
n_ads_watched                            -0.0013      0.000    -10.476      0.000      -0.002      -0.001
n_battlepass_lvls_finished                0.0484      0.004     13.389      0.000       0.041       0.055
n_extra_challenge_buys                   -0.0808      0.013     -6.053      0.000      -0.107      -0.055
connect_fb_attempt                        0.0122      0.007      1.775      0.076      -0.001       0.026
n_accept_all_friends                     -0.0176      0.007     -2.505      0.012      -0.031      -0.004
n_accept_friend                          -0.0557      0.010     -5.772      0.000      -0.075      -0.037
n_clicked_add_friends                    -0.0134      0.004     -3.516      0.000      -0.021      -0.006
n_missions_completed                      0.0004   6.44e-05      5.543      0.000       0.000       0.000
n_package_info_offers_viewed             -0.0001   9.76e-06    -11.432      0.000      -0.000   -9.24e-05
n_payments_package_key                   -0.0198      0.006     -3.473      0.001      -0.031      -0.009
sum_payments_package_key                  0.0040      0.000     24.196      0.000       0.004       0.004
n_package_tips_offers_viewed              0.0012      0.000      4.157      0.000       0.001       0.002
n_remove_ad_clicks                        0.0143      0.007      2.140      0.032       0.001       0.027
n_sessions_ended                         -0.0061      0.001     -4.412      0.000      -0.009      -0.003
total_session_duration                 5.575e-09   3.53e-10     15.798      0.000    4.88e-09    6.27e-09
max_session_end_viplevel              -3.787e-13   1.19e-13     -3.185      0.001   -6.12e-13   -1.46e-13
n_sessions_started                        0.0057      0.001      4.206      0.000       0.003       0.008
min_session_start_viplevel                0.3810      0.120      3.182      0.001       0.146       0.616
min_session_start_player_level            0.0180      0.003      6.568      0.000       0.013       0.023
n_time_skip_buys                         -0.0033      0.001     -2.330      0.020      -0.006      -0.001
n_viplevels_completed                     0.2790      0.008     32.927      0.000       0.262       0.296
bigmac_dollar_price                      -0.0110      0.003     -3.957      0.000      -0.017      -0.006
gdp_per_capita                         1.031e-05   2.53e-07     40.770      0.000    9.82e-06    1.08e-05
ram_m

In [46]:
adj_for_log_X_test = X_test.drop(["n_ad_reward_fails","n_instant_awards_claims","connected_fb","n_friend_searches","n_levels_completed","n_calendar_login_days","rated_game","max_session_end_player_level","screen_density","avg_country_spend_30d","n_friend_installs"],axis=1)
adj_for_log_X_train = X_train.drop(["n_ad_reward_fails","n_instant_awards_claims","connected_fb","n_friend_searches","n_levels_completed","n_calendar_login_days","rated_game","max_session_end_player_level","screen_density","avg_country_spend_30d","n_friend_installs"],axis=1)
log_regr = linear_model.LinearRegression()
log_regr.fit(adj_for_log_X_train,log_y_train)
log_y_pred = log_regr.predict(adj_for_log_X_test)

In [48]:
print(f"R^2 skóre pre trénovací dataset: {log_regr.score(adj_for_log_X_train,log_y_train)}")
print(f"R^2 skóre pre testovací dataset: {log_regr.score(adj_for_log_X_test,log_y_test)}")
print(f"MAE: {mean_absolute_error(log_y_test,log_y_pred)}")
print(f"MSE: {mean_squared_error(log_y_test,log_y_pred)}")

R^2 skóre pre trénovací dataset: 0.4857734905808355
R^2 skóre pre testovací dataset: 0.48647918701665516
MAE: 0.9117657207651216
MSE: 1.3648123876732323
